In [1]:
import torch
CUDA = torch.cuda.is_available()
print(CUDA)

False


In [2]:
import time
from lifelines.utils import concordance_index 
import sys
from torch import nn
import survival_analysis_chirag
import numpy as np
import pandas as pd
import network
from torch.utils.data import TensorDataset, Dataset
import torch.utils.data.dataloader as dataloader
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


ds = pd.read_csv('whas500.csv',sep=',')
train = ds[:400]
validation = ds[400:]

x = train[['age', 'gender', 'bmi', 'chf', 'miord']]
e = train['fstat']
t = train['lenfol']

x = torch.from_numpy(x.as_matrix()).float()
e = torch.from_numpy(e.as_matrix()).float()
t = torch.from_numpy(t.as_matrix())

In [3]:
if CUDA:
    x = x.cuda()
    e = e.cuda()
    t = t.cuda()

In [4]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
#         m.weight.data.fill_(0)
#         m.bias.data.fill_(1)

def init_weights_for_cox(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0)
        m.bias.data.fill_(0)

In [5]:
def train(model, x, e, t, CUDA, optimizer, n_epochs):

    # Initialize Metrics
    c_index = []
    valid_c_index = []

    risk_set = []
    for i in range(len(t)):
        temp = []
        for j in range(len(t)):
            if t[j] >= t[i]:
                temp.append(j)
        risk_set.append(temp)
    
    start = time.time()
    for epoch in range(n_epochs):

        optimizer.zero_grad()
        # print("x: ", x)
        outputs = model(x)

        loss = negative_log_likelihood(outputs, e, risk_set, CUDA)
        loss.backward()
        optimizer.step()


        print(loss.cpu().data.numpy())
        
        ci_train = get_concordance_index(outputs, t, e)
        c_index.append(ci_train)
        torch.cuda.empty_cache()
                   
        print('Finished Training with %d iterations in %0.2fs' % (epoch + 1, time.time() - start))
    
    metrics = {}
    metrics['c-index'] = c_index
    return metrics


In [6]:
def negative_log_likelihood(risk, E, risk_set, CUDA):
    new_risk = []
    for i in range(len(risk_set)):
        new_risk.append(risk[risk_set[i]])
        
    log_risk = []
    for i in range(len(new_risk)):
        temp = torch.logsumexp(new_risk[i], 0)
        log_risk.append(temp)

    uncensored_likelihood = torch.transpose(risk, 0, 1) - torch.tensor(log_risk)
    censored_likelihood = uncensored_likelihood * E
    num_observed_events = torch.sum(E)
    neg_likelihood = -torch.sum(censored_likelihood)# / num_observed_events
    return neg_likelihood


In [7]:
def get_concordance_index(x, t, e, **kwargs):
    x = x.detach().cpu().numpy()
    t = t.detach().cpu().numpy()
    e = e.detach().cpu().numpy()
    computed_hazard = np.exp(x)

    return concordance_index(t,-1*computed_hazard,e)


In [22]:

# For CPH, set cox argument as True
print("CPH model")
n_in = x.shape[1]

layers_sizes = [n_in, 48, 48, 1]

# Construct Neural Network
layers = []
for i in range(len(layers_sizes)-2):
    layers.append(nn.Linear(layers_sizes[i],layers_sizes[i+1]))
    layers.append(nn.ReLU())

layers.append(nn.Linear(layers_sizes[-2], layers_sizes[-1]))
my_network = nn.Sequential(*layers)
my_network.apply(init_weights)

#optimizer = optimizer = torch.optim.SGD(my_network.parameters(), lr=learning_rate, momentum=momentum, weight_decay=L2_reg, nesterov=True)

optimizer = torch.optim.Adam(my_network.parameters(), lr=1e-4)
my_network.train()
if CUDA:
    my_network.cuda()

# If you have validation data, you can add it as the valid_dataloader parameter to the function
n_epochs = 50
metrics = train(my_network, x, e, t, CUDA, optimizer, n_epochs)
print()

print("Done")

CPH model
1135.8035
Finished Training with 1 iterations in 0.02s
1133.6571
Finished Training with 2 iterations in 0.04s
1131.5813
Finished Training with 3 iterations in 0.07s
1129.4191
Finished Training with 4 iterations in 0.09s
1127.3672
Finished Training with 5 iterations in 0.11s
1125.4402
Finished Training with 6 iterations in 0.13s
1123.613
Finished Training with 7 iterations in 0.15s
1121.9609
Finished Training with 8 iterations in 0.19s
1120.4762
Finished Training with 9 iterations in 0.20s
1119.0332
Finished Training with 10 iterations in 0.22s
1117.5244
Finished Training with 11 iterations in 0.26s
1116.1117
Finished Training with 12 iterations in 0.29s
1114.8391
Finished Training with 13 iterations in 0.31s
1113.7388
Finished Training with 14 iterations in 0.33s
1112.7856
Finished Training with 15 iterations in 0.37s
1112.0177
Finished Training with 16 iterations in 0.41s
1111.431
Finished Training with 17 iterations in 0.42s
1111.0688
Finished Training with 18 iterations in

In [41]:
my_network

Sequential(
  (0): Linear(in_features=5, out_features=48, bias=True)
  (1): ReLU()
  (2): Linear(in_features=48, out_features=48, bias=True)
  (3): ReLU()
  (4): Linear(in_features=48, out_features=1, bias=True)
)

In [15]:
print("Train C-Index:", metrics['c-index'])

Train C-Index: [0.2703059339410057, 0.272579630118354, 0.2750157331655129, 0.27830447227917743, 0.2824255466006212, 0.2871759475425811, 0.2919872510607199, 0.2990925516149333, 0.3064820641913153, 0.3148866197040135, 0.32398140441340667, 0.3349438681256217, 0.346738667045616, 0.36046204754461114, 0.3753222761322804, 0.3917456708418766, 0.409184108487789, 0.42576991006719583, 0.4435737631701821, 0.46005806045595726, 0.4750400941959845, 0.4893928013154956, 0.503258287825575, 0.5163929434215068, 0.5299742179094176, 0.5428652632006334, 0.5533405063034166, 0.5629225116222416, 0.5715706774396557, 0.5791428977445746, 0.5860248888527985, 0.5926226679388538, 0.5996264641994357, 0.6048437848921009, 0.6106904322052823, 0.6159077528979476, 0.6212671796016972, 0.625266448770783, 0.6290627093526057, 0.6328183682169756, 0.6358634970259242, 0.6388274223999675, 0.6415477374692949, 0.6433342130372115, 0.6458515195192757, 0.6476988976633712, 0.6493432672202034, 0.6508861324834041, 0.6521650865831625, 0.65